In [1]:
import pandas as pd
import numpy as np
import pandas_ta as ta
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error 
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dropout, Flatten, Dense

In [2]:

# Step 1: Data cleaning
data = yf.download(tickers='BTC-usd', period='max', interval='1d')
data.dropna(inplace=True)
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...
2023-01-20,21085.373047,22692.357422,20919.126953,22676.552734,22676.552734,28799154319
2023-01-21,22677.427734,23282.347656,22511.833984,22777.625000,22777.625000,32442278429
2023-01-22,22777.986328,23056.730469,22387.900391,22720.416016,22720.416016,24746386230


In [3]:
# Step 2: Preprocess the data
# Normalize the prices
data = (data - np.min(data)) / (np.max(data) - np.min(data))
data.drop([ 'Close'], axis=1, inplace=True)
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3052 entries, 2014-09-17 to 2023-01-24
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       3052 non-null   float64
 1   High       3052 non-null   float64
 2   Low        3052 non-null   float64
 3   Adj Close  3052 non-null   float64
 4   Volume     3052 non-null   float64
dtypes: float64(5)
memory usage: 143.1 KB


c:\Users\pc\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
c:\Users\pc\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


,Open,High,Low,Adj Close,Volume
Date,,,,,
2014-09-17,0.004289,0.003739,0.004243,0.004144,0.000043
2014-09-18,0.004155,0.003574,0.003649,0.003655,0.000081
2014-09-19,0.003669,0.003151,0.003217,0.003216,0.000091
2014-09-20,0.003232,0.003085,0.003298,0.003425,0.000088
2014-09-21,0.003431,0.002927,0.003348,0.003275,0.000059


In [4]:
# Split the data into inputs and outputs
x = data.drop(["Adj Close"], axis=1)
y = data["Adj Close"]
x.info()
y.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3052 entries, 2014-09-17 to 2023-01-24
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    3052 non-null   float64
 1   High    3052 non-null   float64
 2   Low     3052 non-null   float64
 3   Volume  3052 non-null   float64
dtypes: float64(4)
memory usage: 119.2 KB
<class 'pandas.core.series.Series'>
DatetimeIndex: 3052 entries, 2014-09-17 to 2023-01-24
Series name: Adj Close
Non-Null Count  Dtype  
--------------  -----  
3052 non-null   float64
dtypes: float64(1)
memory usage: 47.7 KB


In [5]:
# Step 3: Split the data into training and testing sets

split = int(len(data) * 0.8)
X_train, X_test = x[:split], x[split:]
y_train, y_test = y[:split], y[split:]
print("x_test", X_test.shape)
print("y_test",y_test.shape)
print("X_train",X_train.shape)
print("y_train", y_train.shape)

x_test (611, 4)
y_test (611,)
X_train (2441, 4)
y_train (2441,)


In [8]:
# Step 5: Reshaping the data
x_train = np.reshape(X_train, (X_train.shape[0],1, X_train.shape[1]))
x_test = np.reshape(X_test, (X_test.shape[0],1, X_test.shape[1]))
print('X_train', X_train.shape)
print('y_train', y_train.shape)
print('X_test', X_test.shape)
print('Y_test', y_test.shape) 
print('X_train.shape[1]', X_train.shape[1])
#print('X_train.shape[2]', X_train.shape[2])




ValueError: Must pass 2-d input. shape=(2441, 1, 4)

In [13]:
# Step 6: Building the CNN model (cont.)
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(1, 4)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv1d. Consider increasing the input size. Received input shape [None, 1, 4] which would produce output shape with a zero or negative value in a dimension.

In [10]:
# Step 7: Training and evaluating the model
history = model.fit(X_train, y_train, epochs=250, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/250
77/77 [==============================] - 6s 18ms/step - loss: 0.0343 - val_loss: 0.2238
Epoch 2/250
77/77 [==============================] - 1s 9ms/step - loss: 0.0292 - val_loss: 0.2052
Epoch 3/250
77/77 [==============================] - 1s 9ms/step - loss: 0.0291 - val_loss: 0.2050
Epoch 4/250
77/77 [==============================] - 1s 10ms/step - loss: 0.0291 - val_loss: 0.2033
Epoch 5/250
77/77 [==============================] - 1s 9ms/step - loss: 0.0291 - val_loss: 0.2025
Epoch 6/250
77/77 [==============================] - 1s 11ms/step - loss: 0.0291 - val_loss: 0.2053
Epoch 7/250
77/77 [==============================] - 1s 11ms/step - loss: 0.0290 - val_loss: 0.2022
Epoch 8/250
77/77 [==============================] - 1s 11ms/step - loss: 0.0291 - val_loss: 0.2043
Epoch 9/250
77/77 [==============================] - 1s 9ms/step - loss: 0.0290 - val_loss: 0.2038
Epoch 10/250
77/77 [==============================] - 1s 9ms/step - loss: 0.0290 - val_loss: 0.2046
Epoc

In [11]:
# Evaluate the model on the test set
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score:.4f}')

Test loss: 0.0009


In [13]:

# Step 8: Tuning the Hyperparameters
# You can use techniques such as grid search or random search to find the best values for the model's hyperparameters

# Step 9: Testing the model
y_pred = model.predict(X_test)
print('y_pred',y_pred.shape)
print('y_test',y_test.shape)
#y_pred = y_pred.reshape(-1,1)
#y_pred = np.nan_to_num(y_pred)
#y_pred = scaler.inverse_transform(y_pred)
#y_pred = scaler.inverse_transform(y_pred[:,[-1]])
#y_test = scaler.inverse_transform(y_test.reshape(-1, 1))
#y_test = np.nan_to_num(y_test)


#evaluat
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = 100 * np.mean(np.abs((y_test - y_pred) / y_test))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("MAPE: {:.4f}%".format(mape))
print("RMSE: {:.4f}".format(rmse))
print("MSE: {:.4f}".format(mse))
print("MAE:{:.4f}".format(mae))

20/20 [==============================] - 0s 5ms/step
y_pred (611, 1)
y_test (611, 1)
MAPE: 4.1675%
RMSE: 0.0306
MSE: 0.0009
MAE:0.0230


In [14]:
def smape(y_true, y_pred):
    return 200 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true)))

SMAPE1 = smape(y_test, y_pred)
print("smape: {:.4f}%".format(SMAPE1))

smape: 4.1606%


In [17]:
import matplotlib.pyplot as plt

# Assume that you have stored the date information in a variable called 'date'
# Assume that you have stored the predictions in a variable called 'y_pred'
# Assume that you have stored the test data in a variable called 'y_test'

# Plot the test data and predictions on a line chart
plt.plot(data['Date'], y_pred, label='Predictions')
plt.plot(data['Date'], y_test, label='Test Data')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

KeyError: 'Date'

In [20]:
y_pred = scaler.inverse_transform(y_pred)
#y_pred = scaler.inverse_transform(y_pred[:,[-1]])
#y_test = scaler.inverse_transform(y_test.reshape(-1, 1))
print ('y_pred',y_pred)

ValueError: non-broadcastable output operand with shape (611,1) doesn't match the broadcast shape (611,4)